<a href="https://colab.research.google.com/github/Jeevan-Neupane/Deep-Learning/blob/main/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import torch


## Data Preprocessing

In [2]:
torch.__version__

'2.5.1+cu121'

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')


In [4]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
X= dataset.iloc[:,3:-1].values
X


array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [7]:
y=dataset.iloc[:,-1].values
y

array([1, 0, 1, ..., 1, 1, 0])

## Encoding Categorical DAta

In [9]:
from sklearn.preprocessing import LabelEncoder
le =LabelEncoder();
X[:,2]=le.fit_transform(X[:,2])

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [11]:
X

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

## Splitting the dataset into the training set and test set

In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=42);

## Feature Scaling

In [13]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np

# Sample dataset (replace with your actual dataset)
# Assuming X_train, y_train, X_test, y_test are already defined and are NumPy arrays.

# Preprocess data: Scaling using StandardScaler
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)  # Fit and transform training data
X_test_scaled = sc.transform(X_test)  # Only transform the test data

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Define the ANN model
class ANN(nn.Module):
    def __init__(self):
        super(ANN, self).__init__()
        self.fc1 = nn.Linear(in_features=X_train_tensor.shape[1], out_features=6)  # 6 units in the first layer
        self.fc2 = nn.Linear(in_features=6, out_features=1)  # 1 output unit
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))  # Apply ReLU activation after the first layer
        x = self.sigmoid(self.fc2(x))  # Apply Sigmoid activation at the output layer
        return x

# Create the model instance
model = ANN()

# Define the loss function (Binary Cross-Entropy) and optimizer (Adam)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

# Create a DataLoader for batching
batch_size = 32
train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

# Training the model
epochs = 100
for epoch in range(epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()  # Zero the gradients

        # Forward pass
        outputs = model(inputs)

        # Calculate the loss
        loss = criterion(outputs.squeeze(), labels)

        # Backward pass (calculate gradients)
        loss.backward()

        # Update weights
        optimizer.step()

        running_loss += loss.item()

    # Print loss for each epoch
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(train_loader)}")

# Set the model to evaluation mode for prediction
model.eval()

# Make predictions on the test set
with torch.no_grad():
    y_pred = model(X_test_tensor)

# Apply the threshold of 0.5 to convert probabilities to binary labels
y_pred_binary = (y_pred.squeeze() > 0.5).float()

# Convert the predictions and true labels to NumPy arrays for evaluation
y_pred_binary = y_pred_binary.numpy()
y_test = y_test_tensor.numpy()

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_binary)
print("Confusion Matrix:\n", cm)

# Accuracy Score
accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy Score:", accuracy)


Epoch [1/100], Loss: 0.705900246143341
Epoch [2/100], Loss: 0.5142231810092927
Epoch [3/100], Loss: 0.4597560778856277
Epoch [4/100], Loss: 0.43764593636989596
Epoch [5/100], Loss: 0.42149833077192306
Epoch [6/100], Loss: 0.40601176548004153
Epoch [7/100], Loss: 0.39246914130449295
Epoch [8/100], Loss: 0.3817962634563446
Epoch [9/100], Loss: 0.37380549949407577
Epoch [10/100], Loss: 0.3677825582623482
Epoch [11/100], Loss: 0.36357975959777833
Epoch [12/100], Loss: 0.3599926142692566
Epoch [13/100], Loss: 0.35718732011318205
Epoch [14/100], Loss: 0.35452788305282595
Epoch [15/100], Loss: 0.3529411650896072
Epoch [16/100], Loss: 0.35115996944904326
Epoch [17/100], Loss: 0.3499519422054291
Epoch [18/100], Loss: 0.3485164502263069
Epoch [19/100], Loss: 0.34744799554347994
Epoch [20/100], Loss: 0.346392995595932
Epoch [21/100], Loss: 0.3456579444408417
Epoch [22/100], Loss: 0.3450154703259468
Epoch [23/100], Loss: 0.34477113336324694
Epoch [24/100], Loss: 0.34421232467889784
Epoch [25/100],